In [54]:
#Testing the Oracle database
import imp
import cx_Oracle
import pandas as pd
import numpy as np

#Read in Oracle database
dsn = (cx_Oracle.makedsn('sanger-sb-prod.ca4xce6yzabf.eu-west-1.rds.amazonaws.com', port = '1521', service_name='MOSAIC'))
connection = cx_Oracle.connect(user='MOSAIC_USER', password='*Of7!LW9auUKfooS',dsn=dsn)
cursor = connection.cursor()

print(cursor)

#Grab all the Barcode IDs from the google spreadsheet
file_path =  "/lustre/scratch126/humgen/projects/sc-eqtl-ibd/analysis/marcus_analysis/IBD-R Sample Record - PBMC_Isolation_and_scRNAseq.csv"
data = pd.read_csv(file_path, header=1)

data = data.rename(columns={
    data.columns[1]: 'Blood_tube_Barcode',
    data.columns[2]: 'Latest_date_for_processing_BLOOD_(yyyy-mm-dd)'
})

data = data.dropna(subset=['Participent_Study_ID'])

participant_study_ids = data['Blood_tube_Barcode'].tolist()
participant_study_ids_str = ', '.join(participant_study_ids)
result_str = '"' + ','.join(["'" + item.strip() + "'" for item in participant_study_ids_str.split(", ")]) + '"'
result_str = '"' + ','.join(result_str.strip('"').split(',')[2:]) + '"'
result_str = result_str.replace('"', '')
cursor.execute(f'''
    SELECT 
        A."Barcode",
        A."Substance ID",
        B."SITE",
        E."Receipt Temperature"  -- The Receipt Temperature column
    FROM mosaic.container_report A 
    LEFT JOIN (
        SELECT 
            mosaic.substance_report."Name",
            mosaic.substance_report."Substance ID",
            mosaic.substance_report."Property Value" AS "SITE"
        FROM mosaic.substance_report 
        WHERE mosaic.substance_report."Property Name" LIKE 'Site'
    ) B ON B."Substance ID" LIKE A."Barcode" OR B."Substance ID" LIKE A."Substance ID"
    LEFT JOIN (  -- Join for "Receipt Temperature"
        SELECT 
            mosaic.substance_report."Name",
            mosaic.substance_report."Substance ID",
            mosaic.substance_report."Property Value" AS "Receipt Temperature"
        FROM mosaic.substance_report 
        WHERE mosaic.substance_report."Property Name" LIKE 'Receipt Temperature'
    ) E ON E."Substance ID" LIKE A."Barcode" OR E."Substance ID" LIKE A."Substance ID"
    WHERE A."Barcode" IN ({result_str})
''')

Reviewed_metadata_donors_barcodes = pd.DataFrame(cursor.fetchall())

<cx_Oracle.Cursor on <cx_Oracle.Connection to MOSAIC_USER@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=sanger-sb-prod.ca4xce6yzabf.eu-west-1.rds.amazonaws.com)(PORT=1521))(CONNECT_DATA=(SERVICE_NAME=MOSAIC)))>>


NameError: name 'df' is not defined

In [55]:
Reviewed_metadata_donors_barcodes.to_csv('temperature_barcoded.csv', index=False)  